# Bài tập phần Khai phá luật kết hợp (thầy Hiếu) <br>
Học viên: Lê Trung Hiếu <br>
Mã HV: 20007920<br>
Ngày sinh: 16/08/1992

In [1]:
import re
import csv
import pandas as pd
import numpy as np
from lxml import etree
from vncorenlp import VnCoreNLP
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def cleaning(text):
    # remove text written between double curly braces
    article_txt = re.sub(r"{{.*}}","",text)
    
    # remove file attachments
    article_txt = re.sub(r"\[\[File:.*\]\]","",article_txt)
    
    # remove Image attachments
    article_txt = re.sub(r"\[\[Image:.*\]\]","",article_txt)
    
    # remove unwanted lines starting from special characters
    article_txt = re.sub(r"\n: \'\'.*","",article_txt)
    article_txt = re.sub(r"\n!.*","",article_txt)
    article_txt = re.sub(r"^:\'\'.*","",article_txt)
    
    # remove non-breaking space symbols
    article_txt = re.sub(r"&nbsp","",article_txt)
    
    # remove URLs link
    article_txt = re.sub(r"http\S+","",article_txt)
    
    # remove digits from text
    article_txt = re.sub(r"\d+","",article_txt)
    
    # remove text written between small braces
    article_txt = re.sub(r"\(.*\)","",article_txt)
    
    # remove sentence which tells category of article
    article_txt = re.sub(r"Category:.*","",article_txt)
    
    # remove the sentences inside infobox or taxobox
    article_txt = re.sub(r"\| .*","",article_txt)
    article_txt = re.sub(r"\n\|.*","",article_txt)
    article_txt = re.sub(r"\n \|.*","",article_txt)
    article_txt = re.sub(r".* \|\n","",article_txt)
    article_txt = re.sub(r".*\|\n","",article_txt)
    
    # remove infobox or taxobox
    article_txt = re.sub(r"{{Infobox.*","",article_txt)
    article_txt = re.sub(r"{{infobox.*","",article_txt)
    article_txt = re.sub(r"{{taxobox.*","",article_txt)
    article_txt = re.sub(r"{{Taxobox.*","",article_txt)
    article_txt = re.sub(r"{{ Infobox.*","",article_txt)
    article_txt = re.sub(r"{{ infobox.*","",article_txt)
    article_txt = re.sub(r"{{ taxobox.*","",article_txt)
    article_txt = re.sub(r"{{ Taxobox.*","",article_txt)
    
    # remove lines starting from *
    article_txt = re.sub(r"\* .*","",article_txt)
        
    # remove new line character
    article_txt = re.sub(r"\n","",article_txt)
    
    article_txt = re.sub(r"<ref.+ref>","",article_txt)
    article_txt = re.sub(r"<div.+div>","",article_txt)
    article_txt = re.sub(r"<td.+td>","",article_txt)
    article_txt = re.sub(r"<tr.+tr>","",article_txt)  
    
    # replace all punctuations with space
    article_txt = re.sub(r"\!|\"|\#|\$|\%|\&|\'|\(|\)|\*|\.|\:|\+|\,|\-|\/|\;|\|\?|\@|\[|\\|\]|\^|\_|\`|\{|\||\}|\~"," ",article_txt)
    
    # replace consecutive multiple space with single space
    article_txt = re.sub(r" +"," ",article_txt)
    
    # replace non-breaking space with regular space
    article_txt = article_txt.replace(u'\xa0', u' ')
    return article_txt.strip().lower()

In [3]:
class Wikipedia:
    def __init__(self, fh, tag):
        """
        Initialize 'iterparse' to only generate 'end' events on tag '<entity>'
        """
        # Prepend the default Namespace {*} to get anything.
        self.context = etree.iterparse(fh, events=("end",), tag=['{*}' + tag])

    def _parse(self):
        """
        Parse the XML File for all '<tag>...</tag>' Elements
        Clear/Delete the Element Tree after processing
        """
        for event, elem in self.context:
            yield event, elem

            elem.clear()
            while elem.getprevious() is not None:
                del elem.getparent()[0]

    def __iter__(self):
        """
        Iterate all '<tag>...</tag>' Element Trees yielded from self._parse()
        :return: Dict var 'entity' {tag1, value, tag2, value, ... ,tagn, value}}
        """
        for event, elem in self._parse():
            entity = {}
            # Assign the 'elem.namespace' to the 'xpath'
            ele_text = elem.xpath('./xmlns:revision/xmlns:text/text()', namespaces={'xmlns':etree.QName(elem).namespace})
            text = ele_text[0] if ele_text else ""
            entity['text'] = cleaning(text[:text.find("==")])
            entity['title'] = elem.xpath('./xmlns:title/text()', namespaces={'xmlns':etree.QName(elem).namespace})[0]
            yield entity


In [4]:
df = pd.read_csv("viwiki.csv")

In [ ]:
annotator = VnCoreNLP("VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g')
vectorizer = CountVectorizer(
    analyzer='word',
    ngram_range=(3, 3)
)

In [7]:
arr = []
for i in range(0, df.shape[0], 5000):
    working_df = df[i:i+5000]
    working_df.loc[:, "tokenized"] = working_df["text"].apply(lambda x: " ".join(annotator.tokenize(x)[0]))
    X = vectorizer.fit_transform(working_df["tokenized"])

    word_list = vectorizer.get_feature_names()
    count_list = X.toarray().sum(axis=0)

    d = dict(zip(word_list,count_list))
    d = {k: v for k, v in zip(word_list, count_list) if "_" in k and v > 5}
    arr.append(d)

In [8]:
result = {}

for d in arr:
    for key, value in d.items():
        if key in result:
            result[key] += value
        else:
            result[key] = value

sorted_result = dict(sorted(result.items(), key=lambda item: item[1], reverse=True))
sorted_result

{'khoa_học đầu_tiên năm': 311854,
 'loài thực_vật có': 262612,
 'thực_vật có hoa': 255994,
 'một loài thực_vật': 244847,
 'mô_tả khoa_học đầu_tiên': 234126,
 'miêu_tả khoa_học đầu_tiên': 103326,
 'thông_tin khu dân_cư': 96306,
 'miêu_tả khoa_học năm': 74167,
 'bọ cánh_cứng trong': 68765,
 'cánh_cứng trong họ': 68714,
 'loài bọ cánh_cứng': 66779,
 'dân_số thời_điểm năm': 43467,
 'thời_điểm năm là': 43240,
 'là một phân_cấp': 41459,
 'một phân_cấp hành_chính': 41454,
 'phân_cấp hành_chính thổ_nhĩ_kỳ': 36978,
 'thổ_nhĩ_kỳ dân_số thời_điểm': 35815,
 'bảng thông_tin thành_phố': 34991,
 'thông_tin thành_phố pháp': 34862,
 'hành_chính thổ_nhĩ_kỳ xã': 34547,
 'thổ_nhĩ_kỳ xã thuộc': 34547,
 'là một đô_thị': 32543,
 'có diện_tích km²': 29261,
 'một loài bộ_cánh': 29028,
 'tiểu hành_tinh vành_đai': 28327,
 'diện_tích km² dân_số': 25081,
 'là loài thực_vật': 24886,
 'dương_xỉ trong họ': 22834,
 'một loài dương_xỉ': 22819,
 'loài dương_xỉ trong': 22816,
 'mô_tả khoa_học năm': 21808,
 'danh_pháp kho